### Importing Libraries

In [1]:
#Importing libraries
import numpy as np
import pandas as pd
import cv2 
import pytesseract as pt
from PIL import Image
pt.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract"
import nltk

In [2]:
import spacy
nlp = spacy.load('en_core_web_sm',disable=['ner','textcat'])
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to C:\Users\ROSHINI R
[nltk_data]     S\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\ROSHINI R
[nltk_data]     S\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ROSHINI R S\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest

### OCR

In [4]:
#OCR
def img_txt(img_path):
    global text
    img = cv2.imread(img_path)
    img = cv2.resize(img, None, fx = 1,fy = 1)
    kernel = np.ones((1, 1), np.uint8)
    img = cv2.dilate(img, kernel, iterations=1)
    img = cv2.erode(img, kernel, iterations=1)
    blurred=cv2.GaussianBlur(img,(5,5),0)
    edged=cv2.Canny(blurred,30,50)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    adpt = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY, 85,10)
    text = pt.image_to_string(adpt)
    text = text.replace('\n',' ')
    print(text)
    cv2.imshow("adaptive threshold",adpt)
    cv2.waitKey(0)

### Getting Input from the user

In [ ]:
#inupt image through webcam
cam = cv2.VideoCapture(0)

cv2.namedWindow("Capture")

img_counter = 0

while True:
    ret, frame = cam.read()
    if not ret:
        print("failed to grab frame")
        break
    
    cv2.imshow("Capture", frame)
 
    k = cv2.waitKey(1)
    if k%256 == 27:
        # ESC pressed
        print("Escape hit, closing...")
        break
    elif k%256 == 32:
        # SPACE pressed  
        img_name = "opencv_frame_{}.png".format(img_counter)
        cv2.imwrite(img_name, frame)
        print("{} written!".format(img_name))
        img_counter += 1
    
cam.release()
cv2.destroyAllWindows()

In [5]:
#Load the image from local file
img_name = 'inputimg.jpg'

### Display raw text from the Image

In [6]:
#Process the image and print text
print("Scanned text:\n")
img_txt(img_name)

Scanned text:

   Artificial intelligence (Al) makes it possible for machines to learn from experience, adjust to new inputs and perform human-like tasks. Most Al examples that you hear about today - from chess-playing computers to self-driving cars - rely heavily on deep learning and natural language processing. Using these technologies, computers can be trained to accomplish specific tasks by processing large amounts of data and recognizing patterns in the data. 


### Summarize the text

### Using Spacy to summarise the text

In [7]:
nlp = spacy.load('en')
doc = nlp(text)
type(doc)
len(list(doc.sents))
keyword = []
stopwords = list(STOP_WORDS)
pos_tag = ['PROPN', 'ADJ', 'NOUN', 'VERB']
for token in doc:
    if(token.text in stopwords or token.text in punctuation):
        continue
    if(token.pos_ in pos_tag):
        keyword.append(token.text)
        
freq_word = Counter(keyword)
print(freq_word.most_common(5))

[('Al', 2), ('tasks', 2), ('computers', 2), ('processing', 2), ('data', 2)]


In [8]:
max_freq = Counter(keyword).most_common(1)[0][1]
for word in freq_word.keys():  
        freq_word[word] = (freq_word[word]/max_freq)
freq_word.most_common(5)

[('Al', 1.0),
 ('tasks', 1.0),
 ('computers', 1.0),
 ('processing', 1.0),
 ('data', 1.0)]

In [9]:
sent_strength={}
for sent in doc.sents:
    for word in sent:
        if word.text in freq_word.keys():
            if sent in sent_strength.keys():
                sent_strength[sent]+=freq_word[word.text]
            else:
                sent_strength[sent]=freq_word[word.text]
# print(sent_strength)

In [10]:
summarized_sentences = nlargest(4, sent_strength, key=sent_strength.get)
# print(summarized_sentences)

In [11]:
final_sentences = [ w.text for w in summarized_sentences ]
summary = ' '.join(final_sentences)
print("Summarized text: \n")
print(summary)

Summarized text: 

Most Al examples that you hear about today - from chess-playing computers to self-driving cars - rely heavily on deep learning and natural language processing. Using these technologies, computers can be trained to accomplish specific tasks by processing large amounts of data and recognizing patterns in the data.     Artificial intelligence (Al) makes it possible for machines to learn from experience, adjust to new inputs and perform human-like tasks.


### Extracting key phrases from the summary

In [12]:
import RAKE
import operator
stop_dir = r"C:\Users\ROSHINI R S\anaconda3\Lib\site-packages\RAKE\stoplists\SmartStopList.txt"
rake_object=RAKE.Rake(stop_dir)
def Sort_Tuple(tup):
    tup.sort(key=lambda x: x[1])
    return tup
keywords = Sort_Tuple(rake_object.run(summary))[-5:]
print("Keywords: \n",keywords)

Keywords: 
 [('chess-playing computers', 8.0), ('accomplish specific tasks', 8.0), ('self-driving cars', 9.0), ('natural language processing', 9.0), ('processing large amounts', 9.0)]


In [13]:
Phrase=[]
for i in keywords:
    Phrase.append(i[0])
print("Keyphrases: ")
Phrase

Keyphrases: 


['chess-playing computers',
 'accomplish specific tasks',
 'self-driving cars',
 'natural language processing',
 'processing large amounts']

### Finding relevant links from the keywords

In [ ]:
#Automate google search for getting relevant links
import requests
import urllib.request
import re

try:
    from googlesearch import search
except ImportError:
    print("No module named 'google' found")
Website_link=[]
website_video_link=[]
youtube_link=[]
news_link=[]

for i in Phrase:
    for j in search(i, tld="co.in", num=1, start=0,stop=1,pause=2,safe='on'):
        Website_link.append(j)
    for k in search(i, tld="co.in", num=1, start=0,stop=1,pause=2,tpe="vid",safe='on'):
        website_video_link.append(k)
    for l in search(i, tld="co.in", num=1, start=0,stop=1,pause=2,tpe="nws",safe='on'):
        news_link.append(l)

for m in Phrase:
  query=m.replace(" ","+")
  html=urllib.request.urlopen("https://www.youtube.com/results?search_query="+query)
  video_ids = re.findall(r"watch\?v=(\S{11})", html.read().decode())
  link=("https://www.youtube.com/watch?v=" + video_ids[0])
  youtube_link.append(link)


In [34]:
df = pd.DataFrame(list(zip(Phrase,Website_link,website_video_link,youtube_link,news_link)),columns = ['Key phrase', 'Website Link','website with video link','youtube video link','news link'])
print(df)

                    Key phrase  \
0      chess-playing computers   
1    accomplish specific tasks   
2            self-driving cars   
3  natural language processing   
4     processing large amounts   

                                        Website Link  \
0       https://en.wikipedia.org/wiki/Computer_chess   
1  https://www.1stsource.com/advice/business/sbr_...   
2        https://www.youtube.com/watch?v=tlThdr3O5Qo   
3  https://aliz.ai/natural-language-processing-a-...   
4  https://www.ibm.com/docs/SSZJPZ_9.1.0/com.ibm....   

                             website with video link  \
0       https://en.wikipedia.org/wiki/Computer_chess   
1  https://www.1stsource.com/advice/business/sbr_...   
2        https://www.youtube.com/watch?v=tlThdr3O5Qo   
3  https://aliz.ai/natural-language-processing-a-...   
4  https://www.ibm.com/docs/SSZJPZ_9.1.0/com.ibm....   

                            youtube video link  \
0  https://www.youtube.com/watch?v=h0QQJgiR_A8   
1  https://www.youtub

### Exporting links

In [37]:
filepath = input("Enter filename: ")
filepath = filepath + '.xlsx'
with open(filepath, 'w+') as fp:
    pass
df.to_excel(filepath,index=True)

Enter filename: WWW
